<a href="https://colab.research.google.com/github/DrBharathiTC/Online-Retail-Customer-Segmentation-/blob/main/Online_Retail_Customer_Segmentation_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Extraction/identification of major topics & themes discussed in news articles. </u></b>

## <b> Problem Description </b>

### In this project, your task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## <b> Data Description </b>

### <b>Attribute Information: </b>

* ### InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* ### StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* ### Description: Product (item) name. Nominal.
* ### Quantity: The quantities of each product (item) per transaction. Numeric.
* ### InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
* ### UnitPrice: Unit price. Numeric, Product price per unit in sterling.
* ### CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* ### Country: Country name. Nominal, the name of the country where each customer resides.

In [4]:
import numpy as np
import pandas as pd
from numpy import math
import seaborn as sns
from datetime import datetime
from sklearn.cluster import AgglomerativeClustering

import warnings
from pylab import rcParams
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
orcs = pd.read_excel('/content/drive/My Drive/almabetter projects/Customer Segmentation/Online Retail.xlsx')

In [ ]:
orcs.head(1000)

In [ ]:
orcs.info()

In [ ]:
orcs.shape

In [ ]:
orcs.describe()

### **EDA**


In [ ]:
#checking for null values
orcs.isnull().sum()

In [ ]:
#visualising null values
sns.heatmap(orcs.isnull())

About 24.93% customer IDs are missing in the dataset and 0.27% product descriptions are missing these features cannot be imputed as they are unique for each customer, so we have to drop these variable values.

In [ ]:
orcs.dropna(axis = 0, inplace = True)
orcs.shape

In [ ]:
sns.heatmap(orcs.isnull())

In [ ]:
orcs.isnull().sum()

In [ ]:
orcs.describe()

In [ ]:
#we need to drop produsts which are cancelled from the dataset. Cancelled data can be identified as "InvoiceNo" starting with 'C'.
orcs['InvoiceNo'] = orcs['InvoiceNo'].astype('str')
orcs2=orcs[~orcs['InvoiceNo'].str.contains('C')]

In [ ]:
orcs2.shape


We can split 'InvoiceDate' variable as 'Iyear','Imonth','Iday'and'Ihour'.

In [ ]:
orcs2["Invoice_Year"] = orcs2['InvoiceDate'].dt.year
orcs2['Invoice_Month'] = orcs2['InvoiceDate'].dt.strftime('%B')
orcs2['Invoice_Day'] = orcs2['InvoiceDate'].dt.strftime('%A')
orcs2['Invoice_Hour'] = orcs2['InvoiceDate'].dt.hour

In [ ]:
#We have the 'UnitPrice' and 'Quantity', through this we can calculate the 'TotalExpenditure' of each Invoice.
orcs2['TotalExpenditure']=orcs2['Quantity']*orcs2['UnitPrice'] 

In [ ]:
orcs2.head()

In [ ]:
orcs2.info()

In [ ]:
orcs2.describe()

Lets find out the top

1.customers

2.countries with sales

3.product

In [ ]:
#Top 10 customers
TopCustomers = pd.DataFrame(orcs2['CustomerID'].value_counts().sort_values(ascending = False).reset_index())
TopCustomers.rename(columns = {'index':'CustomerID','CustomerID':'counts'},inplace = True)
plt.figure(figsize = (9,5))
plot1=sns.barplot(x = 'CustomerID',y = 'counts',palette= 'Set2',data = TopCustomers[:10])
plt.xlabel('CustomerID')
plt.ylabel('Frequency')
plt.title("Top 1O Customers.")


In [ ]:
#Top 10 countries
top_countries = pd.DataFrame(orcs2['Country'].value_counts().sort_values(ascending = False).reset_index())
top_countries['Country %']= top_countries['Country']*100/orcs2['Country'].count()
top_countries.rename(columns = {'index':'Country','Country':'TotalCounts'},inplace = True)
fig, ax = plt.subplots(figsize = (15,9))
splot = sns.barplot(x='Country',y='TotalCounts',data = top_countries[:10], lw = 3)
plt.title("Top 1O Customers.")
plt.xlabel('Countries')
plt.ylabel('Count')


In [ ]:
#Top 5 Products
TopProducts = pd.DataFrame(orcs2['Description'].value_counts().sort_values(ascending = False).reset_index())
TopProducts['Description %']= TopProducts['Description']*100/orcs2['Description'].count()
TopProducts.rename(columns = {'index':'Description','Description':'TotalCounts'},inplace = True)
fig, ax = plt.subplots(figsize = (20,9))
plot3 = sns.barplot(x='Description',y='TotalCounts',data = TopProducts[:5], lw = 3)
plt.title('Top 10 Products purchased ')
plt.xlabel('Description')
plt.ylabel('Count')




Top months of purchase

In [ ]:
def PLOT(plot, feature):
    total = len(feature)
    for p in plot.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + p.get_width()/ 10-0.05 
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size = 12)
       
    plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a countplot using Seaborn
ax = sns.countplot(orcs2['Invoice_Month'])

# Set x-axis labels rotation to 90 degrees
plt.xticks(rotation=90)

# Show the modified plot and close any additional empty plots
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a countplot using Seaborn
ax = sns.countplot(orcs2['Invoice_Day'])

# Set x-axis labels rotation to 90 degrees
plt.xticks(rotation=90)


# Show the modified plot and close any additional empty plots
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a countplot using Seaborn
ax = sns.countplot(orcs2['Invoice_Hour'])

# Set x-axis labels rotation to 90 degrees
plt.xticks(rotation=90)


# Show the modified plot and close any additional empty plots
plt.show()

In [ ]:
# Distribution of total expenditure
plt.figure(figsize=(15,10))
sns.distplot(np.log1p(orcs2['TotalExpenditure']))

RFM Segmentation
Recency,Frequencyand Monetary(RFM)is a technique to evaluate customers based on their transactions.The evaluation is based on;

How recent their last transaction was(Recency)?

How many transactions they have made in the last year(Frequency)?

What was the value of their transaction was(Monetary)?


In [ ]:
#Calculating RFM scores
import datetime as dt
LD = dt.datetime(2011,12,10)
#Creating RFM scores for each customer
RFM = orcs2.groupby('CustomerID').agg({'InvoiceDate': lambda x: (LD - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalExpenditure': lambda x: x.sum()})

#Converting Invoice Date into type int
RFM['InvoiceDate'] = RFM['InvoiceDate'].astype(int)

#Rename column names to Recency, Frequency and Monetary
RFM.rename(columns={'InvoiceDate': 'Recency', 
                         'InvoiceNo': 'Frequency', 
                         'TotalExpenditure': 'Monetary'}, inplace=True)

RFM.reset_index().head()


In [ ]:
RFM.info()

In [ ]:
RFM.shape

In [ ]:
#RFM descriptives
RFM.describe()

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(RFM['Recency'])

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(RFM['Frequency'])

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(RFM['Monetary'])

In [ ]:
#log trnsformation
RFM['Recency'] = np.log(RFM['Recency']+1) 
RFM['Frequency'] = np.log(RFM['Frequency']+1)
RFM['Monetary'] = np.log(RFM['Monetary']+1)

In [ ]:
fig, axis = plt.subplots(nrows=1, ncols= 3, figsize=(20,10))
sns.histplot(RFM['Recency'], ax=axis[0])
sns.histplot(RFM['Frequency'], ax=axis[1])
sns.histplot(RFM['Monetary'], ax=axis[2])
fig.suptitle("Data distribution after log transform", size=25)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# plot data points in 3D space
fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)
x = RFM['Recency']
y = RFM['Frequency']
z = RFM['Monetary']
ax.scatter(x, y, z, marker='.')
ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')
plt.title("Data Visualization", size=35)

In [ ]:
RFM.shape

Deciding on Optimal CLUSTERS

Hierachial clustering

It is used for taking decision on number of clusters.

In [ ]:
rcParams['figure.figsize'] = 30, 15
HC = AgglomerativeClustering(affinity = 'euclidean', linkage = 'ward')
yHC = HC.fit_predict(RFM)

In [ ]:
import scipy.cluster.hierarchy as sch
dendrogram = sch.dendrogram(sch.linkage(RFM, method = 'ward'))
plt.title('Dendrogram',fontweight = 'bold')
plt.xlabel('Customers',fontweight = 'bold')
plt.ylabel('Euclidean Distances',fontweight = 'bold')
plt.show()

Dendrogram is usually subjective as it differs based on viewer. However, we can specifically determine the clusters through a euclidean distance based threshold value. I am keeping the Threshold cut off of 50 euclidean.

In [ ]:
max_d = 50
dendrogram = sch.dendrogram(sch.linkage(RFM, method = 'ward'))
plt.axhline(y = max_d,c = 'k')
plt.title('Dendrogram',fontweight = 'bold')
plt.xlabel('Customers',fontweight = 'bold')
plt.ylabel('Euclidean Distances',fontweight = 'bold')
plt.show()

Based on the cut-off we can decide to select 4 optimal clusters.

K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
#elbow plot
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,15), timings= True)
visualizer.fit(RFM)        # Fit data to visualizer
visualizer.show()

The elbow point is achieved with 5 clusters which is highlighted by the function itself. The function also informs us about how much time was needed to plot models for various numbers of clusters through the green line.

In [ ]:
# Silhouette Score for K means
# Import ElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,15),metric='silhouette', timings= True)
visualizer.fit(RFM)        # Fit the data to the visualizer
visualizer.show()  

The Silhouette Coefficient(SC) tells us if individual points are correctly assigned to their clusters. We can use the following thumb rules while using Silhouette Coefficient:

1.SC close to 0 means that the point is between two clusters.

2.If it is closer to -1, then we would be better off assigning it to the other clusters.

3.If SC is close to 1, then the point belongs to the ‘correct’ cluster.

Here we can see the optimal clusters through siloutte method is 2.

In [ ]:
# Calinski Harabasz Score for K means
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,15),metric='calinski_harabasz', timings= True)
visualizer.fit(RFM)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure  

Calinski Harabasz Index is ideal when number of clusters is 2 for the K means clustering algorithm.

In [ ]:
# Davies Bouldin score for K means
from sklearn.datasets import load_iris
from sklearn.metrics import davies_bouldin_score

In [ ]:
iris = load_iris()
RFM = iris.data[:, :2]

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=30)
labels = kmeans.fit_predict(RFM)

In [ ]:
db_index = davies_bouldin_score(RFM, labels)
print(db_index)

In [ ]:
results = {}

for i in range(2,11):
    kmeans = KMeans(n_clusters=i, random_state=30)
    labels = kmeans.fit_predict(RFM)
    db_index = davies_bouldin_score(RFM, labels)
    results.update({i: db_index})

In [ ]:
plt.plot(list(results.keys()), list(results.values()))
plt.xlabel("Number of clusters")
plt.ylabel("Davies-Boulding Index")
plt.show()

Davies Bouldin score is minimized with 3 clusters and can be considered for the k means algorithm



In [ ]:
# BIC for GMM
from sklearn.mixture import GaussianMixture
n_components = range(1, 15)
covariance_type = ['spherical', 'tied', 'diag', 'full']
BICscore=[]
for cov in covariance_type:
    for n_comp in n_components:
        gmm=GaussianMixture(n_components=n_comp,covariance_type=cov)
        gmm.fit(RFM)
        BICscore.append((cov,n_comp,gmm.bic(RFM)))
BICscore

The lower the BIC score, better is the model.try out various combinations of the parameters and select the model with the lowest BIC score. Based on low average BIC scores on we can choose the optimal clusters to be 3 .

# **SUMMARY**

In [ ]:
dictr = {"Model_Name":["Hierachicalclustering-Dendrogram ","KMeans-Distortion score Elbow method  ", "KMeans-Silhouette","KMeans-Calinski Harbasz score","KMeans-Davies Bouldin score","BIC-GMM "],
  "Data":["RFM", "RFM", "RFM","RFM","RFM","RFM"],
  "Optimal_Number_of_cluster":[4,6,2,2,3,3]}

In [ ]:
Summary = pd.DataFrame(dictr)

In [ ]:
Summary.head(6)

Multiple techniques to detect optimal clusters shows varied results. We can take mean value(3.33) as the number of clusters,3

K=3 have optimal score. Let's visualize these clusters.



In [ ]:
RFM.shape

In [ ]:
Clusters = KMeans(n_clusters=3, random_state=1)
cluster_labels = Clusters.fit_predict(RFM)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)
x = RFM['Recency']
y = RFM['Frequency']
z = RFM['Monetary']
colors = cm.nipy_spectral(cluster_labels.astype(float) / 3)
ax.scatter(x, y, z, c=colors, marker='.')
ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')
plt.title("Data Visualization", size=25)

In [ ]:
# add cluster label as varable
RFM['Cluster'] = cluster_labels
RFM.head()

In [ ]:
# add cluster label as varable
RFM['Cluster'] = cluster_labels
RFM.head()

In [ ]:
RFMfinal = RFM.groupby('Cluster').mean()

In [ ]:
RFMfinal= RFMfinal.applymap(np.exp)
RFMfinal = RFMfinal.applymap(int)
RFMfinal

In [ ]:
DICTC = {"CLUSTER":["0","1", "2"],
  "Intepretation":["At par recency,frequency and monetary spending", "most recent and frequent transaction and high monetary spending", "No recent or frequent transaction with least monetary spending"],
  "Optimal_Number_of_cluster":['MODERATELY LOYAL CUSTOMERS','LOYAL(TARGET)CUSTOMERS','AT RISK CUSTOMERS']}

In [ ]:
InterpretationSummary = pd.DataFrame(DICTC)

In [ ]:
InterpretationSummary.head()